# Amazon SageMaker Feature Storeの基本的な使い方

## 使用するデータ
[SageMaker Feature Storeのサンプル](https://github.com/aws/amazon-sagemaker-examples/tree/main/sagemaker-featurestore)を利用。
以下の三種類のデータが存在する。

- 顧客データ (feature_store_introduction_customer.csv)
- 更新された顧客データ (feature_store_introduction_customer_updated.csv)
- 注文データ (feature_store_introduction_orders.csv)

In [1]:
import pandas as pd

customer_df = pd.read_csv('s3://machine-learning-workshop/feature-store/feature_store_introduction_customer.csv')
customer_df

,customer_id,city_code,state_code,country_code
0,573291,1,49,2
1,109382,2,40,2
2,828400,3,31,2
3,124013,4,5,2


In [2]:
order_df = pd.read_csv('s3://machine-learning-workshop/feature-store/feature_store_introduction_orders.csv')
order_df

,customer_id,order_id,order_status,store_id
0,573291,4132,1,303
1,109382,5724,0,201
2,828400,1942,0,431
3,124013,6782,1,213


## Feature Groupを作成
２種類のFeature Groupを作成する。
- 顧客用Feature Group
- 注文用Feature Group

In [3]:
import sys

import boto3
import pandas as pd
import numpy as np
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role

role = get_execution_role()

sagemaker_session = sagemaker.Session()
region = sagemaker_session.boto_region_name

In [4]:
import time
from time import strftime, gmtime
customers_feature_group_name = 'customers-feature-group-' + strftime('%d-%H-%M-%S', gmtime())
orders_feature_group_name = 'orders-feature-group-' + strftime('%d-%H-%M-%S', gmtime())

In [5]:
from sagemaker.feature_store.feature_group import FeatureGroup

customers_feature_group = FeatureGroup(
    name=customers_feature_group_name, sagemaker_session=sagemaker_session
)
orders_feature_group = FeatureGroup(
    name=orders_feature_group_name, sagemaker_session=sagemaker_session
)


### データにevent timeを付与
Feature Groupに保存するデータは

- record識別子: データの各レコードを一意に識別するfeature (今回はcustomer_idを利用)
- event time: レコードの作成または更新に対応するfeature (今回はEventTimeというfeatureを作成)

の２つの定義が必要。

#### DataFrameにEvenTimeを付与

In [6]:
import time
current_time_sec = int(round(time.time()))

event_time_col = "EventTime"

customer_df[event_time_col] = pd.Series([current_time_sec]*len(customer_df), dtype="float64")
order_df[event_time_col] = pd.Series([current_time_sec]*len(order_df), dtype="float64")

In [7]:
customers_feature_group.load_feature_definitions(data_frame=customer_df)
orders_feature_group.load_feature_definitions(data_frame=order_df)

[FeatureDefinition(feature_name='customer_id', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='order_id', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='order_status', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='store_id', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>),
 FeatureDefinition(feature_name='EventTime', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>)]

#### レコード識別子を指定して、Feature Groupを作成
Feature Groupを作成するとAmazon Glue Data Catalogも同時に作成される。

In [8]:
prefix = "feature-group-demo"
s3_bucket_name = "machine-learning-workshop"
record_identifier_feature_name = "customer_id"


customers_feature_group.create(
    s3_uri=f"s3://{s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_col,
    role_arn=role,
    enable_online_store=True,
)

orders_feature_group.create(
    s3_uri=f"s3://{s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_col,
    role_arn=role,
    enable_online_store=True,
)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:980831117329:feature-group/orders-feature-group-29-07-54-05',
 'ResponseMetadata': {'RequestId': '32b17c1a-89c2-47d0-a385-ee44d9f70dfc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '32b17c1a-89c2-47d0-a385-ee44d9f70dfc',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '109',
   'date': 'Sat, 29 Oct 2022 07:54:12 GMT'},
  'RetryAttempts': 1}}

### Feature Groupの中身を確認

In [9]:
customers_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:980831117329:feature-group/customers-feature-group-29-07-54-05',
 'FeatureGroupName': 'customers-feature-group-29-07-54-05',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'customer_id',
   'FeatureType': 'Integral'},
  {'FeatureName': 'city_code', 'FeatureType': 'Integral'},
  {'FeatureName': 'state_code', 'FeatureType': 'Integral'},
  {'FeatureName': 'country_code', 'FeatureType': 'Integral'},
  {'FeatureName': 'EventTime', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2022, 10, 29, 7, 54, 11, 576000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://machine-learning-workshop/feature-group-demo',
   'ResolvedOutputS3Uri': 's3://machine-learning-workshop/feature-group-demo/980831117329/sagemaker/us-east-1/offline-store/customers-feature-group-29-07-54-05-16670300

In [10]:
orders_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:980831117329:feature-group/orders-feature-group-29-07-54-05',
 'FeatureGroupName': 'orders-feature-group-29-07-54-05',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'customer_id',
   'FeatureType': 'Integral'},
  {'FeatureName': 'order_id', 'FeatureType': 'Integral'},
  {'FeatureName': 'order_status', 'FeatureType': 'Integral'},
  {'FeatureName': 'store_id', 'FeatureType': 'Integral'},
  {'FeatureName': 'EventTime', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2022, 10, 29, 7, 54, 12, 470000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://machine-learning-workshop/feature-group-demo',
   'ResolvedOutputS3Uri': 's3://machine-learning-workshop/feature-group-demo/980831117329/sagemaker/us-east-1/offline-store/orders-feature-group-29-07-54-05-1667030052/data'},
 

### Feature Groupのリストを表示

In [11]:
sagemaker_session.boto_session.client('sagemaker', region_name=region).list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'orders-feature-group-29-07-54-05',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:980831117329:feature-group/orders-feature-group-29-07-54-05',
   'CreationTime': datetime.datetime(2022, 10, 29, 7, 54, 12, 470000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Creating'},
  {'FeatureGroupName': 'customers-feature-group-29-07-54-05',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:980831117329:feature-group/customers-feature-group-29-07-54-05',
   'CreationTime': datetime.datetime(2022, 10, 29, 7, 54, 11, 576000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Creating'}],
 'ResponseMetadata': {'RequestId': '1757fd45-68ab-47be-9a50-506d317997c0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1757fd45-68ab-47be-9a50-506d317997c0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '488',
   'date': 'Sat, 29 Oct 2022 07:54:18 GMT'},
  'RetryAttempts': 0}}

In [12]:
!aws sagemaker list-feature-groups

{
    "FeatureGroupSummaries": [
        {
            "FeatureGroupName": "orders-feature-group-29-07-54-05",
            "FeatureGroupArn": "arn:aws:sagemaker:us-east-1:980831117329:feature-group/orders-feature-group-29-07-54-05",
            "CreationTime": 1667030052.47,
            "FeatureGroupStatus": "Creating"
        },
        {
            "FeatureGroupName": "customers-feature-group-29-07-54-05",
            "FeatureGroupArn": "arn:aws:sagemaker:us-east-1:980831117329:feature-group/customers-feature-group-29-07-54-05",
            "CreationTime": 1667030051.576,
            "FeatureGroupStatus": "Creating"
        }
    ]
}


### 既存のFeature Groupを呼び出す
Feature Group名を使えば既存のFeature Groupは簡単に呼び出せる

In [13]:
existing_customers_feature_group = FeatureGroup(
    name=customers_feature_group_name, sagemaker_session=sagemaker_session
)

In [14]:
existing_customers_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:980831117329:feature-group/customers-feature-group-29-07-54-05',
 'FeatureGroupName': 'customers-feature-group-29-07-54-05',
 'RecordIdentifierFeatureName': 'customer_id',
 'EventTimeFeatureName': 'EventTime',
 'FeatureDefinitions': [{'FeatureName': 'customer_id',
   'FeatureType': 'Integral'},
  {'FeatureName': 'city_code', 'FeatureType': 'Integral'},
  {'FeatureName': 'state_code', 'FeatureType': 'Integral'},
  {'FeatureName': 'country_code', 'FeatureType': 'Integral'},
  {'FeatureName': 'EventTime', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2022, 10, 29, 7, 54, 11, 576000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': 's3://machine-learning-workshop/feature-group-demo',
   'ResolvedOutputS3Uri': 's3://machine-learning-workshop/feature-group-demo/980831117329/sagemaker/us-east-1/offline-store/customers-feature-group-29-07-54-05-16670300

## データを保存

In [15]:
customers_feature_group.ingest(
    data_frame=customer_df, max_workers=3, wait=True
)

IngestionManagerPandas(feature_group_name='customers-feature-group-29-07-54-05', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fc9b12f09d0>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7fc9b177e450>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

In [16]:
orders_feature_group.ingest(
    data_frame=order_df, max_workers=3, wait=True
)

IngestionManagerPandas(feature_group_name='orders-feature-group-29-07-54-05', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fc9b12f09d0>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7fc9b17644d0>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

### Online Storeからの個別のデータの取得

In [17]:
customer_id = 573291
sagemaker_session.boto_session.client(
    'sagemaker-featurestore-runtime',
    region_name=region).get_record(FeatureGroupName=customers_feature_group_name,
                                   RecordIdentifierValueAsString=str(customer_id))

{'ResponseMetadata': {'RequestId': 'cb301a49-b31e-4548-ba96-1cd858e9289b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cb301a49-b31e-4548-ba96-1cd858e9289b',
   'content-type': 'application/json',
   'content-length': '275',
   'date': 'Sat, 29 Oct 2022 07:54:44 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'customer_id', 'ValueAsString': '573291'},
  {'FeatureName': 'city_code', 'ValueAsString': '1'},
  {'FeatureName': 'state_code', 'ValueAsString': '49'},
  {'FeatureName': 'country_code', 'ValueAsString': '2'},
  {'FeatureName': 'EventTime', 'ValueAsString': '1667030049.0'}]}

### Online Storeからの複数データの取得

In [18]:
records = sagemaker_session.boto_session.client(
    "sagemaker-featurestore-runtime", region_name=region
).batch_get_record(
    Identifiers=[
        {
            "FeatureGroupName": customers_feature_group_name,
            "RecordIdentifiersValueAsString": ["573291", "109382", "828400", "124013"],
        },
        {
            "FeatureGroupName": orders_feature_group_name,
            "RecordIdentifiersValueAsString": ["573291", "109382", "828400", "124013"],
        },
    ]
)
records

{'ResponseMetadata': {'RequestId': 'b6775686-b874-46ce-b5c7-0f25ee03b6bb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'b6775686-b874-46ce-b5c7-0f25ee03b6bb',
   'content-type': 'application/json',
   'content-length': '3036',
   'date': 'Sat, 29 Oct 2022 07:54:48 GMT'},
  'RetryAttempts': 0},
 'Records': [{'FeatureGroupName': 'customers-feature-group-29-07-54-05',
   'RecordIdentifierValueAsString': '573291',
   'Record': [{'FeatureName': 'customer_id', 'ValueAsString': '573291'},
    {'FeatureName': 'city_code', 'ValueAsString': '1'},
    {'FeatureName': 'state_code', 'ValueAsString': '49'},
    {'FeatureName': 'country_code', 'ValueAsString': '2'},
    {'FeatureName': 'EventTime', 'ValueAsString': '1667030049.0'}]},
  {'FeatureGroupName': 'customers-feature-group-29-07-54-05',
   'RecordIdentifierValueAsString': '109382',
   'Record': [{'FeatureName': 'customer_id', 'ValueAsString': '109382'},
    {'FeatureName': 'city_code', 'ValueAsString': '2'},
    {'FeatureNa

### データの削除

In [19]:
customer_id = 573291
sample_record = sagemaker_session.boto_session.client(
    'sagemaker-featurestore-runtime',
    region_name=region).delete_record(FeatureGroupName=customers_feature_group_name,
                                      RecordIdentifierValueAsString=str(customer_id),
                                      EventTime=str(current_time_sec))

In [20]:
customer_id = 573291
sagemaker_session.boto_session.client(
    'sagemaker-featurestore-runtime',
    region_name=region).get_record(FeatureGroupName=customers_feature_group_name,
                                   RecordIdentifierValueAsString=str(customer_id))

{'ResponseMetadata': {'RequestId': 'd2ced7fa-1a0e-42f7-9a26-3b00bf3b4a5e',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd2ced7fa-1a0e-42f7-9a26-3b00bf3b4a5e',
   'content-type': 'application/json',
   'content-length': '15',
   'date': 'Sat, 29 Oct 2022 07:54:53 GMT'},
  'RetryAttempts': 0}}

### クエリによるOffline Storeからのデータの取得

In [21]:
customer_query = customers_feature_group.athena_query()
customer_table = customer_query.table_name
customer_table

'customers-feature-group-29-07-54-05-1667030051'

In [22]:
query_string = f'SELECT * FROM "{customer_table}"'
customer_query.run(query_string=query_string, output_location=f"s3://{s3_bucket_name}/query_results/")
customer_query.wait()

In [23]:
customer_query.as_dataframe()

,customer_id,city_code,state_code,country_code,eventtime,write_time,api_invocation_time,is_deleted
0,124013,4.0,5.0,2.0,1.667030e+09,2022-10-29 08:00:25.424,2022-10-29 07:54:38.000,False
1,573291,1.0,49.0,2.0,1.667030e+09,2022-10-29 08:00:25.414,2022-10-29 07:54:38.000,False
2,573291,NaN,NaN,NaN,1.667030e+09,2022-10-29 08:00:25.414,2022-10-29 07:54:52.000,True
3,828400,3.0,31.0,2.0,1.667030e+09,2022-10-29 08:00:25.422,2022-10-29 07:54:38.000,False
4,109382,2.0,40.0,2.0,1.667030e+09,2022-10-29 08:00:25.429,2022-10-29 07:54:38.000,False


## Featureを追加する

In [24]:
from sagemaker.feature_store.feature_definition import StringFeatureDefinition

customers_feature_group.update(
    feature_additions=[StringFeatureDefinition("email"), StringFeatureDefinition("name")]
)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:980831117329:feature-group/customers-feature-group-29-07-54-05',
 'ResponseMetadata': {'RequestId': '7b76ec75-01ab-4b04-ae6b-71ee7b9c4a5b',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '7b76ec75-01ab-4b04-ae6b-71ee7b9c4a5b',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '112',
   'date': 'Sat, 29 Oct 2022 08:39:03 GMT'},
  'RetryAttempts': 0}}

In [25]:
updated_customer_df = pd.read_csv("s3://machine-learning-workshop/feature-store/feature_store_introduction_customer_updated.csv")
updated_customer_df

,customer_id,city_code,state_code,country_code,email,name
0,573291,1,49,2,john.lee@gmail.com,John Lee
1,109382,2,40,2,olivequil@gmail.com,Olive Quil
2,828400,3,31,2,liz.knee@gmail.com,Liz Knee
3,124013,4,5,2,eileenbook@gmail.com,Eileen Book


In [26]:
current_time_sec = int(round(time.time()))
updated_customer_df[event_time_col] = pd.Series([current_time_sec]*len(customer_df), dtype="float64")
customers_feature_group.ingest(
    data_frame=updated_customer_df, max_workers=3, wait=True
)

IngestionManagerPandas(feature_group_name='customers-feature-group-29-07-54-05', sagemaker_fs_runtime_client_config=<botocore.config.Config object at 0x7fc9b12f09d0>, max_workers=3, max_processes=1, profile_name=None, _async_result=<multiprocess.pool.MapResult object at 0x7fc9b197abd0>, _processing_pool=<pool ProcessPool(ncpus=1)>, _failed_indices=[])

In [27]:
customer_id = 573291
sagemaker_session.boto_session.client(
    'sagemaker-featurestore-runtime',
    region_name=region).get_record(FeatureGroupName=customers_feature_group_name,
                                   RecordIdentifierValueAsString=str(customer_id))

{'ResponseMetadata': {'RequestId': '620e90aa-156c-438c-bb66-69c7f73964dc',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '620e90aa-156c-438c-bb66-69c7f73964dc',
   'content-type': 'application/json',
   'content-length': '386',
   'date': 'Sat, 29 Oct 2022 08:40:43 GMT'},
  'RetryAttempts': 0},
 'Record': [{'FeatureName': 'customer_id', 'ValueAsString': '573291'},
  {'FeatureName': 'city_code', 'ValueAsString': '1'},
  {'FeatureName': 'state_code', 'ValueAsString': '49'},
  {'FeatureName': 'country_code', 'ValueAsString': '2'},
  {'FeatureName': 'EventTime', 'ValueAsString': '1667032839.0'},
  {'FeatureName': 'email', 'ValueAsString': 'john.lee@gmail.com'},
  {'FeatureName': 'name', 'ValueAsString': 'John Lee'}]}

In [29]:
query_string = f'SELECT * FROM "{customer_table}"'
customer_query = customers_feature_group.athena_query()
customer_query.run(query_string=query_string, output_location=f"s3://{s3_bucket_name}/query_results/")
customer_query.wait()
customer_query.as_dataframe()

,customer_id,city_code,state_code,country_code,eventtime,email,name,write_time,api_invocation_time,is_deleted
0,124013,4.0,5.0,2.0,1.667033e+09,eileenbook@gmail.com,Eileen Book,2022-10-29 08:45:35.799,2022-10-29 08:40:39.000,False
1,828400,3.0,31.0,2.0,1.667033e+09,liz.knee@gmail.com,Liz Knee,2022-10-29 08:45:35.831,2022-10-29 08:40:38.000,False
2,109382,2.0,40.0,2.0,1.667033e+09,olivequil@gmail.com,Olive Quil,2022-10-29 08:45:35.851,2022-10-29 08:40:39.000,False
3,573291,1.0,49.0,2.0,1.667033e+09,john.lee@gmail.com,John Lee,2022-10-29 08:45:35.984,2022-10-29 08:40:38.000,False
4,828400,3.0,31.0,2.0,1.667030e+09,NaN,NaN,2022-10-29 08:00:25.422,2022-10-29 07:54:38.000,False
5,124013,4.0,5.0,2.0,1.667030e+09,NaN,NaN,2022-10-29 08:00:25.424,2022-10-29 07:54:38.000,False
6,109382,2.0,40.0,2.0,1.667030e+09,NaN,NaN,2022-10-29 08:00:25.429,2022-10-29 07:54:38.000,False
7,573291,1.0,49.0,2.0,1.667030e+09,NaN,NaN,2022-10-29 08:00:25.414,2022-10-29 07:54:38.000,False
8,573291,NaN,NaN,NaN,1.667030e+09,NaN,NaN,2022-10-29 08:00:25.414,2022-10-29 07:54:52.000,True


### Offline Storeからの最新のデータを取得
Offline Storeで最新のsnapshotを手に入れるためには、クエリによる制御が必要となる。

参考: https://docs.aws.amazon.com/sagemaker/latest/dg/feature-store-athena-glue-integration.html

In [30]:
query_string = f"""
SELECT *
FROM
    (SELECT *,
         row_number()
        OVER (PARTITION BY {record_identifier_feature_name}
    ORDER BY {event_time_col} desc, Api_Invocation_Time DESC, write_time DESC) AS row_num
    FROM "{customer_table}")
WHERE row_num = 1;
"""
customer_query.run(query_string=query_string, output_location=f"s3://{s3_bucket_name}/query_results/")
customer_query.wait()
customer_query.as_dataframe()

,customer_id,city_code,state_code,country_code,eventtime,email,name,write_time,api_invocation_time,is_deleted,row_num
0,109382,2,40,2,1.667033e+09,olivequil@gmail.com,Olive Quil,2022-10-29 08:45:35.851,2022-10-29 08:40:39.000,False,1
1,573291,1,49,2,1.667033e+09,john.lee@gmail.com,John Lee,2022-10-29 08:45:35.984,2022-10-29 08:40:38.000,False,1
2,124013,4,5,2,1.667033e+09,eileenbook@gmail.com,Eileen Book,2022-10-29 08:45:35.799,2022-10-29 08:40:39.000,False,1
3,828400,3,31,2,1.667033e+09,liz.knee@gmail.com,Liz Knee,2022-10-29 08:45:35.831,2022-10-29 08:40:38.000,False,1


## Featureへメタデータを付与

In [32]:
customers_feature_group.update_feature_metadata(
    feature_name="customer_id",
    description="カスタマーの識別番号",
)

{'ResponseMetadata': {'RequestId': '9dfd53fb-fa6e-42ca-8ab2-e160eaf3f770',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '9dfd53fb-fa6e-42ca-8ab2-e160eaf3f770',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 29 Oct 2022 13:58:39 GMT'},
  'RetryAttempts': 0}}

In [33]:
customers_feature_group.describe_feature_metadata(feature_name="customer_id")

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:980831117329:feature-group/customers-feature-group-29-07-54-05',
 'FeatureGroupName': 'customers-feature-group-29-07-54-05',
 'FeatureName': 'customer_id',
 'FeatureType': 'Integral',
 'CreationTime': datetime.datetime(2022, 10, 29, 7, 54, 11, 576000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 10, 29, 13, 58, 40, 841000, tzinfo=tzlocal()),
 'Description': 'カスタマーの識別番号',
 'Parameters': [],
 'ResponseMetadata': {'RequestId': '93cca27c-be44-4698-a613-6ad3df4f465d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '93cca27c-be44-4698-a613-6ad3df4f465d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '353',
   'date': 'Sat, 29 Oct 2022 13:58:42 GMT'},
  'RetryAttempts': 0}}

In [34]:
from sagemaker.feature_store.inputs import FeatureParameter

customers_feature_group.update_feature_metadata(
    feature_name="customer_id",
    parameter_additions=[FeatureParameter("idType", "primaryKey")],
)

{'ResponseMetadata': {'RequestId': '44496ef0-4185-489f-84db-923af79adc94',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '44496ef0-4185-489f-84db-923af79adc94',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 29 Oct 2022 13:58:45 GMT'},
  'RetryAttempts': 0}}

In [35]:
customers_feature_group.describe_feature_metadata(feature_name="customer_id")

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:980831117329:feature-group/customers-feature-group-29-07-54-05',
 'FeatureGroupName': 'customers-feature-group-29-07-54-05',
 'FeatureName': 'customer_id',
 'FeatureType': 'Integral',
 'CreationTime': datetime.datetime(2022, 10, 29, 7, 54, 11, 576000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2022, 10, 29, 13, 58, 46, 753000, tzinfo=tzlocal()),
 'Description': 'カスタマーの識別番号',
 'Parameters': [{'Key': 'idType', 'Value': 'primaryKey'}],
 'ResponseMetadata': {'RequestId': '424ad06c-6dbc-485f-a4f0-f1715127678c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '424ad06c-6dbc-485f-a4f0-f1715127678c',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '390',
   'date': 'Sat, 29 Oct 2022 13:58:49 GMT'},
  'RetryAttempts': 0}}

In [36]:
customers_feature_group.delete()

In [37]:
orders_feature_group.delete()